DELTA LAKE - Compaction
- Process of combining many smaller files into few larger files to improve performance and optimize storage 



WHY we need to perform compaction?
- small file problem 
- In delta lake, for every write operation it will create new parquet files, overtime this will create multiple smaller files results in the small file problem for storage and degrade the query performance

Summary : Instaed of having many smaller files, having few larrger files results in the much better performance and efficient storage

There are two types of compaction offered by data bricks
- 1.OPTIMIZE command :
-      This merges multiple small files inot fewer larger files, thus improves performance.
-      When you run the OPTIMIZE command you can specify 1 or more columns to cluster or group the data by while cerating the larger files , you do that on the ZORDER CLAUSE

- 2. Z-ORDER Compaction(ZORDER BY Clause):
-       Duinrg the compaction, delta lake will ensure that specific Z-ORDER column values are kept together in the same file, this informaion is also maintained in the metadata
- 3. LIQUID CLUSTEING



1.OPTIMIZE COMMAND 

create table - demo.delta_lake.optimize_stock_price

In [0]:
%sql
drop table if exists demo.delta_lake.optimize_stock_prices;

create table if not exists demo.delta_lake.optimze_stock_prices
(
  stock_id string,
  price double,
  trading_date date
);

In [0]:
%sql
INSERT INTO demo.delta_lake.optimze_stock_prices
values ('AAPL',227.65,'2025-02-10')

In [0]:
%sql
INSERT INTO demo.delta_lake.optimze_stock_prices
values ('GOOGL',247.65,'2021-02-10')

In [0]:
%sql
INSERT INTO demo.delta_lake.optimze_stock_prices
values ('MSFT',117.65,'2020-02-10')

In [0]:
%sql
INSERT INTO demo.delta_lake.optimze_stock_prices
values ('AMZN',327.65,'2025-10-10')

In [0]:
%sql
DESCRIBE HISTORY demo.delta_lake.optimze_stock_prices;

To check the number of files required to get the latest data

In [0]:
%sql
describe detail demo.delta_lake.optimze_stock_prices;

In [0]:
%sql
select * from  demo.delta_lake.optimze_stock_prices;

see the below picture it has created four different files, so this is not good for the performance

![tavles.png](./tavles.png "tavles.png")

RUN OPTIMIZE 

In [0]:
%sql
OPTIMIZE demo.delta_lake.optimze_stock_prices;

In [0]:
%sql
DESCRIBE HISTORY demo.delta_lake.optimze_stock_prices;

- Ater running the optimize command we didnt remove any hsitory or the existing data 
-it just added another parquet file , with all the  optimized data 
-and while reading it will just ignore all the rest of the files, just read only one file 

Note : you can refer all the points in the below 2 images 

![parquet_l.png](./parquet_l.png "parquet_l.png")

![log.png](./log.png "log.png")

In [0]:
%sql
select * from demo.delta_lake.optimze_stock_prices version as of 3;

Z-ORDER

In [0]:
%sql
OPTIMIZE demo.delta_lake.optimze_stock_prices
ZORDER BY stock_id;

The Z-ORDER has no effect on this data , as we have only one large file , incase if we have multiple larger files , it will cluster of group the data based on the column that u did ZORDER

3. LIQUID CLUSTERING 
- Liquid Clustering is an innovative data layout technique in Delta Lake that automatically manages how data is organized to improve performance and cost efficiency
- It repalces the old techniques like OPTIMIZE, ZORDER
-  Liquid Clustering is automatic way to imporve performace

How the liquid Clusteriing works ?
- When you enable liquid clustering , data is divided into clusters based on the clustering keys 
- These clusters are automatically managed and adjusted as new data arrives
- no manual re-organization or  Z-orderiing required 
- changing the clustering columns doesnt require re-writing entire data, beacuse liquid clustering reclusters incrementally, only the portions of the table that needs imporvement are rewritten.
- Liquid Clustering is dynamic and self maintaining


LIQUID CLUSTERING BENEFITS :
- Faster queries
- Automatic optimization
- Adaptive Data Layout
- Lower maintenance cost 
- simpler operations

LIQUID CLUSTERING - IMPLEMENTATION

In [0]:
%sql
CREATE TABLE TABL1(
  COL1 INT, 
  COL2 STRING
)
CLUSTER BY (COL1)

HOW TO ENABLE LIQUID CLUSTERING ON EXISTING TABLE USING ALTER 

In [0]:
%sql
ALTER TABLE TABLE
CLUSTER BY (col1);

#### DELTA LAKE - VACUUM 
- used to remove old, unused files from the delta lake to free up storage , permanently deletes files that are no longer referenced in the transaction log and older than retention threshold(Default 7 days)

BENEFITS -
- Reduces cloud storage costs 
- Improves query performance
- Helps comply with privacy regulations

1. Check the table hsitory 

In [0]:
%sql
describe history demo.delta_lake.optimze_stock_prices;

BEFORE VACUUM 
![parquet_l.png](./parquet_l.png "parquet_l.png")

2.RUN VACCUM 

In [0]:
%sql
SET spark.databricks.delta.retentionDurationCheck.enabled = false;
VACUUM demo.delta_lake.optimze_stock_prices
RETAIN 0 HOURS;

In [0]:
%sql
describe history demo.delta_lake.optimze_stock_prices;

AFTER VACUUM 
![after_vacuum.png](./after_vacuum.png "after_vacuum.png")

In [0]:
%sql
select * from demo.delta_lake.optimze_stock_prices;

In [0]:
%sql
select * from demo.delta_lake.optimze_stock_prices version as of 3;

since we have deleted the history , it is throwing the error 

![optimize.png](./optimize.png "optimize.png")

# When to use what 
![scenriaso.png](./scenriaso.png "scenriaso.png")